# MBA em Ciência de Dados
## Técnicas Avançadas de Captura e Tratamento de Dados


### <span style="color:darkred">Dados não estruturados: sinais e imagens</span>

### <span style="color:darkred">Avaliação</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

#### <span style="color:red">As respostas devem ser dadas no Moodle, o notebook deve ser usado para executar código para obtenção dos resultados.</span>

---

In [70]:
# carregando as bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Questão 1)

Escolha a alternativa que descreva dados estruturados:

(a) valores obtidos de um sensor de aceleração de um celular a uma taxa de 100 Hz<br>
(b) texto obtido a partir da execução de optical character recognition de um documento PDF<br>
(c) uma imagem da face de uma pessoa armazenada numa base de dados de uma empresa e usada para verificação biométrica<br>
**(d) maior e menor preços referente ao ID de um produto, obtidos de uma base de dados histórica de um e-commerce**<br>
(e) array de pixels de uma imagem obtida da folha de uma planta em um cultivo agrícola por uma câmera digital<br>


### Questão 2)

É um exemplo de dado não estruturado do tipo sequencial e temporal:

(a) os valores de quilômetros de congestionamento estimados em cada bairro da cidade de São Paulo em mesma hora e minuto de um dia<br>
(b) os valores de temperatura coletados num mesmo momento em todas as cidades do estado de São Paulo, ordenadas da maior para a menor<br>
(c) a coluna idade de uma tabela de um banco de dados, ordenada de forma ascendente<br>
(d) os valores de todos os preços de um produto ao longo de um ano obtidos de uma base de dados histórica, ordenados cronologicamente<br>
**(e) o áudio relativo à voz de uma pessoa gravado por celular**<br>

### Questão 3)

Considere as seguintes afirmações sobre o processo de obtenção de embeddings de imagens extraídos de uma base de dados alvo por meio do uso de modelos de redes neurais pré-treinados em um dataset de origem (e diferente do dataset alvo):

I - são obtidos aplicando transformações como a Transformada de Fourier ou Análise de Componentes Principais na base de imagens alvo<br>
II - são características extraídas obtendo a saída da camada de classificação de uma rede neural pré-treinada<br>
III - são características extraídas obtendo a saída de uma camada interna/intermediária de uma rede neural pré-treinada<br>
IV - esse processo assume que o modelo pré-treinado utilizado generaliza para outras tarefas para além do dataset de origem<br>
V - os embeddings resultantes podem ser usado como entrada em algoritmos de aprendizado supervisionado como regressores e classificadores<br>

São verdadeiras as afirmações:

(a) I e II<br>
**(b) II e IV**<br>
(b) II, III e IV<br>
(c) III e IV<br>
**(e) III, IV e V**<br>

### Questão 4)

Carregue os dados do arquivo `pollution_2024.csv` utilizando o comando `pd.read_csv()`

Esse dataframe possui 6 colunas (A, B, C, D, E, F) relativas à medição de monóxido de carbono em 6 localizações. Foram feitas medições durante 2 dias a cada 5 minutos em diferentes posições de uma cidade.

Queremos identificar qual das localizações entre: B, C, D, E, F mais se assemelha à A. Para isso, extraia um vetor de características (para cada sinal) composto de 5 valores:
* Entropia da energia (calculada com 6 blocos)
* Entropia da energia (calculada com 12 blocos)
* Entropia espectral (calculada com 6 blocos)
* Entropia espectral (calculada com 12 blocos)
* Dispersão espectral (considere frequência = 0.5Hz)

Compute a distância Euclidiana entre os vetores de características computados e identifique qual sinal (B, C, D, E ou F) é mais próximo de A.

(a) sinal B<br>
(b) sinal C<br>
(c) sinal D<br>
(d) sinal E<br>
**(e) sinal F**<br>

In [71]:
import pandas as pd

data = pd.read_csv('./dados/pollution_2024.csv',)
data.head(3)

,carbon_monoxide_A,carbon_monoxide_B,carbon_monoxide_C,carbon_monoxide_D,carbon_monoxide_E,carbon_monoxide_F
0,49,78,57,57,123,34
1,45,81,54,54,124,32
2,48,86,59,59,126,27


In [72]:
def entropia_energia(sinal, n_blocos=10):
    '''Entropia da energia do sinal'''
    # energia total
    energia_sinal = np.sum(sinal ** 2)
    M = len(sinal)

    # calcula janelas dentro do sinal
    M_janelas = int(np.floor(M / n_blocos))
    # verifica se tamanho dos blocos
    # é multiplo do tamanho do sinal
    if M != M_janelas * n_blocos:
        sinal = sinal[0:M_janelas * n_blocos]

    # monta matriz [M_janelas x n_blocos]
    janelas = sinal.reshape(M_janelas, n_blocos, order='F').copy()

    # Computa energias de cada janela (normalizada pela do sinal)
    e_janelas = np.sum(janelas ** 2, axis=0) / (energia_sinal + 0.0001)
    #print(e_janelas)

    # Computa entropia entre energias das janelas
    entropia = -np.sum(e_janelas * np.log2(e_janelas + 0.0001))
    return entropia

def entropia_espectral(sinal, n_blocos=16):
    """Computes the spectral entropy"""

    fft_abs = np.abs(np.fft.fft(sinal))

    entropia_esp = entropia_energia(fft_abs, n_blocos=n_blocos)

    return entropia_esp

def centroide_dispersao_espectral(sinal, tx_amostragem):
    '''Calcula o centro de massa e dispersão do espectro do sinal'''

    fft_abs = np.abs( np.fft.fft(sinal) )
    N = len(fft_abs)

    # indices de frequencia
    ind = (np.arange(1, N+1)) * (tx_amostragem / (2.0*N))

    # calcula a distribuicao do espectro normalizando para soma unitária
    Xt = fft_abs.copy()
    Xt = Xt / Xt.max()
    NUM = np.sum(ind * Xt)
    DEN = np.sum(Xt) + 0.0001

    # Centroide:
    centroide = (NUM / DEN)

    # Dispersão:
    dispersao = np.sqrt(np.sum(((ind - centroide) ** 2) * Xt) / DEN)

    # Normalizacao do centroide
    centroide = centroide / (tx_amostragem / 2.0)

    # Normalizando:
    dispersao = dispersao / (tx_amostragem / 2.0)

    return centroide, dispersao

### Questão 5)

Carregue o modelo pré-treinado base MobileNetV2 conforme visto em aula e instruções em: [https://keras.io/api/applications/mobilenet/#mobilenetv2-function]. Use o código disponível no notebook, completando com o código necessário para obter as características de 4 imagens utilizando o modelo de rede neural. Esse modelo deve ter as seguintes camadas: entrada, camada de pré-processamento da MobileNetV2, modelo base MobileNetV2 e camada GlobalAveragePooling2D. Ao carregar o modelo, utilize os parâmetros `include_top=False`, `weights='imagenet'` e o `input_shape` adequado para as imagens que foram carregadas.

Siga os seguintes passos:

1. Obtenha os embeddings a partir do modelo pré-treinado, garantindo que sejam vetores unidimensionais para cada imagem (use reshape se necessário), para obter vetores de características para as imagens;
2. Compute a distância Euclidiana entre o embedding da imagem 0 (primeira imagem carregada) e os embeddings das outras imagens (imagens 1, 2 e 3);
3. Ordene as imagens 1, 2 e 3 da mais próxima para a mais distante da imagem 0, segundo a distância computada no passo anterior.

Qual foi a ordem obtida?

(a) 1, 2, 3 <br>
(b) 1, 3, 2 <br>
(c) 2, 1, 3 <br>
**(d) 2, 3, 1**<br>
(e) 3, 1, 2 <br>

In [73]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import imageio.v2 as imageio

img0 = imageio.imread("dados/texture_dotted.jpg")
img1 = imageio.imread("dados/texture_dotted2.jpg")
img2 = imageio.imread("dados/texture_dotted3.jpg")
img3 = imageio.imread("dados/texture_chequered.jpg")
dataset = np.stack([img0, img1, img2, img3])


In [74]:
dataset[0]

array([[[188,  26,  37],
        [210, 117, 135],
        [238, 179, 183],
        ...,
        [189,  51,  66],
        [195,  72,  90],
        [195,  74,  89]],

       [[190,  31,  49],
        [215, 141, 154],
        [230, 174, 173],
        ...,
        [246, 164, 170],
        [245, 158, 164],
        [235, 160, 157]],

       [[189,  86, 103],
        [230, 174, 177],
        [217, 136, 133],
        ...,
        [195,  86,  89],
        [196,  47,  51],
        [193,  43,  44]],

       ...,

       [[ 52,  35,  28],
        [ 51,  32,  26],
        [ 51,  34,  27],
        ...,
        [ 58,  38,  29],
        [ 58,  35,  27],
        [ 60,  37,  29]],

       [[ 52,  38,  29],
        [ 52,  33,  26],
        [ 54,  35,  28],
        ...,
        [ 98,  85,  76],
        [121, 111, 101],
        [121, 108,  99]],

       [[147, 137, 128],
        [133, 121, 109],
        [ 87,  73,  60],
        ...,
        [198, 193, 187],
        [193, 194, 188],
        [192, 191, 189]]

In [75]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(150, 150, 3),
    include_top=False,
    weights="imagenet"
)


C:\Users\feh_s\AppData\Local\Temp\ipykernel_30864\1561958560.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


In [76]:
# desde a entrada (formato da imagem) até as features de saída
inputs = tf.keras.Input(shape=(150, 150, 3))
x = tf.keras.applications.mobilenet.preprocess_input(inputs)
x = base_model(x)
features = tf.keras.layers.GlobalAveragePooling2D()(x)

# modelo
model_extract_features = tf.keras.Model(inputs, features)


In [77]:
features = model_extract_features.predict(dataset)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [78]:

features

array([[0.13760695, 1.9129977 , 0.        , ..., 0.18433817, 1.9960895 ,
        0.        ],
       [0.01284895, 0.02950228, 0.        , ..., 1.1854535 , 0.03305384,
        0.        ],
       [0.        , 2.5381374 , 0.        , ..., 3.0048265 , 1.3200557 ,
        0.5475383 ],
       [0.        , 1.5047385 , 0.        , ..., 0.09732315, 0.05222915,
        0.        ]], dtype=float32)

In [79]:
features = features.reshape(len(features), -1)
features

array([[0.13760695, 1.9129977 , 0.        , ..., 0.18433817, 1.9960895 ,
        0.        ],
       [0.01284895, 0.02950228, 0.        , ..., 1.1854535 , 0.03305384,
        0.        ],
       [0.        , 2.5381374 , 0.        , ..., 3.0048265 , 1.3200557 ,
        0.5475383 ],
       [0.        , 1.5047385 , 0.        , ..., 0.09732315, 0.05222915,
        0.        ]], dtype=float32)

In [80]:
def distancia(array1, array2):
    distancia_euclidiana = np.linalg.norm(array1 - array2)
    return distancia_euclidiana


In [81]:
print(
    f"Distancia 0 com 1: {distancia(features[0], features[1])}\n",
    f"Distancia 0 com 2: {distancia(features[0], features[2])}\n",
    f"Distancia 0 com 3: {distancia(features[0], features[3])}\n"
    )

Distancia 0 com 1: 37.184383392333984
 Distancia 0 com 2: 29.647626876831055
 Distancia 0 com 3: 33.88526153564453

